In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Load your dataset
# Make sure your CSV file is in the same folder and named 'data.csv'
# Or replace 'data.csv' with your file path.
# The code assumes columns: time, open, high, low, close
df = pd.read_csv('AnhQuocVo/Quant_Research_Strategies/data/BTCUSDm_8Y_M1.csv')

# 2. Preprocess Data
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
df.sort_index(inplace=True)

# 3. Define Strategy Parameters
ORB_MINUTES = 15
RISK_PCT = 0.01  # 1% Risk (Recommended) - NOT 10%
REWARD_RATIO = 2 # 2:1 Reward to Risk

# Identify Unique Days
unique_days = df.index.normalize().unique()

results = []

for day in unique_days:
    # Get data for the current day
    day_data = df[df.index.normalize() == day]
    
    if day_data.empty:
        continue
        
    # Define Session Start (Adjust this based on your market, e.g., 09:30 or 03:00)
    session_start_time = day_data.index[0] # Assumes data starts at open
    orb_end_time = session_start_time + pd.Timedelta(minutes=ORB_MINUTES)
    
    # Get Opening Range Data
    orb_data = day_data[day_data.index < orb_end_time]
    
    if len(orb_data) < 2: # Skip if not enough data
        continue
        
    orb_high = orb_data['high'].max()
    orb_low = orb_data['low'].min()
    orb_mid = (orb_high + orb_low) / 2
    
    # Look for setups AFTER the ORB period
    trading_data = day_data[day_data.index >= orb_end_time]
    
    trade = None
    
    for time, row in trading_data.iterrows():
        # ENTRY LOGIC
        if trade is None:
            if row['close'] > orb_high: # Long Breakout
                entry_price = row['close']
                stop_loss = orb_mid # Stop at midpoint
                risk = entry_price - stop_loss
                target = entry_price + (risk * REWARD_RATIO)
                
                trade = {
                    'Entry Time': time,
                    'Type': 'Long',
                    'Entry Price': entry_price,
                    'Stop Loss': stop_loss,
                    'Target': target,
                    'Result': 0
                }
            elif row['close'] < orb_low: # Short Breakout
                entry_price = row['close']
                stop_loss = orb_mid
                risk = stop_loss - entry_price
                target = entry_price - (risk * REWARD_RATIO)
                
                trade = {
                    'Entry Time': time,
                    'Type': 'Short',
                    'Entry Price': entry_price,
                    'Stop Loss': stop_loss,
                    'Target': target,
                    'Result': 0
                }
        
        # EXIT LOGIC (If in a trade)
        else:
            if trade['Type'] == 'Long':
                if row['low'] <= trade['Stop Loss']:
                    trade['Exit Price'] = trade['Stop Loss']
                    trade['Result'] = -1 # Loss
                    trade['Exit Time'] = time
                    results.append(trade)
                    break
                elif row['high'] >= trade['Target']:
                    trade['Exit Price'] = trade['Target']
                    trade['Result'] = REWARD_RATIO # Win (2R)
                    trade['Exit Time'] = time
                    results.append(trade)
                    break
            elif trade['Type'] == 'Short':
                if row['high'] >= trade['Stop Loss']:
                    trade['Exit Price'] = trade['Stop Loss']
                    trade['Result'] = -1 # Loss
                    trade['Exit Time'] = time
                    results.append(trade)
                    break
                elif row['low'] <= trade['Target']:
                    trade['Exit Price'] = trade['Target']
                    trade['Result'] = REWARD_RATIO # Win (2R)
                    trade['Exit Time'] = time
                    results.append(trade)
                    break
            
            # Time Limit Exit (e.g. close after 30 mins if no result)
            if (time - trade['Entry Time']).total_seconds() > 1800: # 30 mins
                trade['Exit Price'] = row['close']
                trade['Exit Time'] = time
                # Calculate partial PnL (simplified)
                if trade['Type'] == 'Long':
                    trade['Result'] = (row['close'] - trade['Entry Price']) / (trade['Entry Price'] - trade['Stop Loss'])
                else:
                    trade['Result'] = (trade['Entry Price'] - row['close']) / (trade['Stop Loss'] - trade['Entry Price'])
                results.append(trade)
                break


FileNotFoundError: [Errno 2] No such file or directory: 'AnhQuocVo/Quant_Research_Strategies/data/BTCUSDm_8Y_M1.csv'

In [ ]:
# Output Results
results_df = pd.DataFrame(results)
if not results_df.empty:
    print(f"Total Trades: {len(results_df)}")
    print(f"Win Rate: {len(results_df[results_df['Result'] > 0]) / len(results_df) * 100:.2f}%")
    print(f"Total Reward (R-Multiples): {results_df['Result'].sum():.2f}R")
else:
    print("No trades found.")